In [1]:
print('Hello World')
import warnings
warnings.filterwarnings('ignore')

Hello World


In [2]:
!pip install unstructured[pdf] -q


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
%pwd
%cd ..
%pwd

/home/pratham/Desktop/assignment_AIML-INORBVICT


'/home/pratham/Desktop/assignment_AIML-INORBVICT'

In [4]:
import os
documents_folder = "documents"
document_paths = []

for root, dirs, files in os.walk(documents_folder):
    for file in files:
        document_paths.append(os.path.join(root, file))

print(document_paths)

['documents/Managing Diabetes - NIDDK.pdf', 'documents/Diabetes Tests & Diagnosis - NIDDK.pdf', 'documents/Insulin, Medicines, & Other Diabetes Treatments - NIDDK.pdf', 'documents/Symptoms & Causes of Diabetes - NIDDK.pdf', 'documents/Risk Factors for Type 2 Diabetes - NIDDK.pdf', 'documents/Preventing Type 2 Diabetes - NIDDK.pdf', 'documents/Clinical Trials for Diabetes - NIDDK.pdf', 'documents/Diabetes Infographics_0.pdf', 'documents/What Is Diabetes_ - NIDDK.pdf', 'documents/Healthy Living with Diabetes - NIDDK.pdf', 'documents/Preventing Diabetes Problems - NIDDK.pdf', 'documents/Diabetes_ What It Is, Causes, Symptoms, Treatment & Types.pdf']


In [5]:
from langchain_unstructured import UnstructuredLoader
from unstructured.cleaners.core import clean_extra_whitespace

loader = UnstructuredLoader(document_paths,
                            post_processors=[clean_extra_whitespace],
                            chunking_strategy="basic",
                            max_characters=700,
                            chunk_overlap=200)

docs = loader.load_and_split()

docs[5:10]

INFO: pikepdf C++ to Python logger bridge initialized


[Document(metadata={'source': 'documents/Managing Diabetes - NIDDK.pdf', 'file_directory': 'documents', 'filename': 'Managing Diabetes - NIDDK.pdf', 'last_modified': '2025-10-01T16:31:32', 'page_number': 2, 'orig_elements': 'eJzVWG2L4zYQ/isinyOfLMkv2m+lR6GUHoVuKeV6LCNpnKjrl2A7m0uP/veO7GQvm0uPLpTQfNhNRprIGj0zzzPy+08LrLHBdnwIfnHHFrnWaV5Zw4UpkGtvLbc+L3iBRnjprTA+XyzZosERPIxAv/m0cF3X+9DCiMNk17DvtuPDGsNqPdJIqdPEpMaU9MPD3C74cU1TmdGJyYzJaWrThXaMK7x/L00ilyxPVaI/LNnRlCox0cwykZgL9uROA4thP4zYxHh+Ch+x/nkDDhd/0UQVanzwoUc3dv0+OvjObWP8w+Iw3UKDceJHaGEV2hV7G8AihcY4e/f927c/JBtfLaZIhvGh6XyoAk5nJ4XMeCq4SO/T/E6ld0pGvw2s8KHdNhZ78pJxGyN+jOeyoKPoma27zrNVvXXdgMuDuelxGLY92dB65tZdjRRS39WsxiesBxZaNq6R9dCuaGsUUNdQGB49s3s2rbtGqOmM4+P2mymkd9D3MIYnvI/Pp42co29QugxUwS0ozbUucm6gkJQM0pROAQjtroe+kkl6gr6WSR7Nkj6/NCfnW8LeQY9sRGiSU4juw1jjJWiURJmhrHipnOMadcGNrDz3mTKuEjRt5PWgycULaIpsLkSpVKIu2JP7LYHzbY90aAwORbRnDX2yTQ3tq+oJrJJae83RAIFmSgJN2YqnNvWooUSl7PVAMy/qqRCHAorsKb60Z/dbAu23bssa2LNd1/d7YkcY2Rqe4kP98aEEYzuwVRcHd2Fc0+Gxivh2iJT

### diabetes history --> (10000) --> 200 + 500 + 200 + 300 --> adjustent combine docuemnets

In [6]:
len(docs)

220

In [7]:
print(docs[0].page_content)

30/09/2025, 21:19 Managing Diabetes - NIDDK Managing Diabetes On this page: What are the diabetes ABCs? How can a healthy lifestyle help manage my diabetes? What types of medicines should I take? How can I check my blood glucose level at home? How can I keep track of my progress? What special situations should I plan for? Who should be on my health care team? Clinical Trials for Managing Diabetes Diabetes can affect almost every part of your body, including your heart, eyes, kidneys, and nerves. Diabetes is also linked to some types of cancer. You may be able to prevent or delay diabetes health


In [8]:
len(docs[0].page_content)

601

In [9]:
# import getpass
# import os

# if not os.environ.get("GOOGLE_API_KEY"):
#   os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

# from langchain_google_genai import GoogleGenerativeAIEmbeddings

# embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [10]:
# pip install sentence-transformers

In [11]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

INFO: Use pytorch device_name: cpu
INFO: Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


### 768-dimensional vectors & train on 1 Billion sentence pairs

In [12]:
import faiss
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(docs, embedding=embeddings)
vector_store.save_local("faiss_index")

INFO: Loading faiss with AVX512 support.
INFO: Successfully loaded faiss with AVX512 support.


### very large dataset (many millions or more of vectors) and need speed, low latency, and good control over memory.

In [13]:
# import getpass
# import os

# if not os.environ.get("GOOGLE_API_KEY"):
#   os.environ["GOOGLE_API_KEY"] = ''

# from langchain.chat_models import init_chat_model

# model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [14]:
# pip install -q -U google-genai

In [15]:
# from google import genai

# # The client gets the API key from the environment variable `GEMINI_API_KEY`.
# client = genai.Client(api_key='')

# response = client.models.generate_content(
#     model="gemini-2.5-flash", contents="write a one liner on joke windows 11 os "
# )
# print(response.text)

In [16]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
    api_key=''
    # other params...
)

In [17]:
vector_store_local = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

In [18]:
user_question='what is diabetes'

In [19]:
docs = vector_store_local.similarity_search(user_question,k=2)
docs

[Document(id='7f3e5618-ae5c-4393-bde9-6cdd900a0347', metadata={'source': 'documents/What Is Diabetes_ - NIDDK.pdf', 'file_directory': 'documents', 'filename': 'What Is Diabetes_ - NIDDK.pdf', 'last_modified': '2025-10-01T16:31:32', 'page_number': 1, 'orig_elements': 'eJzVWF2L4zYU/Ssiz5FX35bmpS8L7VBYCl0oZbsM+rhOzDhWsJ3dDUv/e6/szDQzG0rnJZA8BJ+rK9vSOffoJp++r6CDHfTTQ5tWd2SlWGJN4jV1wiiqko7UR9HQOgRXcwaN8vVqTVY7mHzyk8c531cx5yG1vZ9gnHHnj/kwPWyh3WwnjFjFK8edszjxNPa1TdMWh7RTldPOGRza57afyh0+fRKqYmvCTaU+r8kJ4feMnK3EazRnIlyNx3GCXVnJb+036H7f+wirv3GgaTt4SO0AccrDsSSkHA9l5ePqNNz7HZSBP7Z+Ivcjed/6ALimB0LJh/v373+t9qlZzWsYp4ddTm3TwrxrgglNOaOMf+TmTvI7KUre3m/goT/sAgyYxctrTPCt7MhKsnfMvSvzcCn8jtuSPx338wv8Aj7hFMx/TY8X+NFMU1cbTVUESW1ygjLmhUsMBI/hGvQ4U5kzfhb4RIliorI/wJuj6PUzfnp6xv+jiillah8iFYI1FK8VtUYyZI55E4wK0DRXoWquD2Eru1A1Q80rOUMhKn4Bz+k3TdY5SR/bqYNLHNUxaAMiUasbT9HagFolAzXW6uCi91LB9Tgyz+U0w9pUrkBtdOUuYHNrBfV0b9KOxJPUjuBHIFN5co7xMIzk6xZ6gjs2kNDlnMimO8Q8wpr4bswk+q6DdBoaDxs/rMutppzJFve+Ij8v6Rg8Z/+DHwY/tV/gY3mPS5Uq

In [20]:
base_prompt = '''
You are a helpful and knowledgeable healthcare chatbot. Your primary function is to provide accurate, general health information, answer common medical questions, and guide users to reliable health resources.
Input during execution will include:
{context}: Any relevant background information or previous conversation turns.
{question}: The user's current specific query.
Key Guidelines for your response to the user's {question} based on {context}:
Informative & Clear: Provide concise, easy-to-understand explanations for medical concepts and symptoms.
Encourage Professional Consultation: Always emphasize the importance of consulting a qualified healthcare professional (doctor, nurse, pharmacist) for personalized medical advice, diagnosis, and treatment. Explicitly state that you cannot provide medical diagnoses or prescribe treatments.
No Specific Recommendations: Refrain from recommending specific medications, treatments, or dosages.
Empathy & Support: Respond with a supportive and empathetic tone.
Structure (as demonstrated in your examples):
Example of JSON Output (for "What are the symptoms of the flu?"):
{{
  "reply": "Common symptoms of the flu typically include fever, cough, sore throat, muscle aches, and fatigue. These symptoms can vary in intensity from person to person, and can often appear suddenly.",
  "guidance_caution": "If you're experiencing these symptoms, especially if they are severe or worsen, it's highly recommended to consult with a healthcare professional for an accurate diagnosis and appropriate advice. They can help determine if it's the flu or another illness.",
  "additional_resource_prompt": "Would you be interested in learning about general flu prevention methods?"
}}

'''

In [21]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

prompt = PromptTemplate(template=base_prompt, input_variables=["context", "question"])

In [22]:
chain = prompt | llm | JsonOutputParser()
response = chain.invoke({"context": docs, "question": user_question})

INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


In [23]:
print(response)

{'reply': 'Diabetes is a chronic condition in which the level of glucose (sugar) in the blood becomes too high. Glucose is the body’s main source of energy and comes from the foods you eat, especially carbohydrates. The hormone insulin – produced by the pancreas – normally helps move glucose from the bloodstream into cells where it can be used for energy. In diabetes, the pancreas either doesn’t make enough insulin, makes none at all, or the body’s cells don’t respond properly to the insulin that is present. As a result, glucose stays in the blood instead of reaching the cells, leading to elevated blood‑sugar levels. Over time, high blood sugar can increase the risk of complications affecting the eyes, kidneys, nerves, heart, and blood vessels.', 'guidance_caution': 'While this overview explains what diabetes is, it does not replace a professional medical evaluation. If you suspect you have diabetes, notice symptoms such as frequent urination, excessive thirst, unexplained weight loss,